##### Topicos

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip install gensim

In [3]:
from gensim import corpora, models, similarities
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import nltk
from nltk.corpus import stopwords

from operator import itemgetter
import re
import pprint

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
!pip install wikipedia

In [5]:
import wikipedia

wikipage = wikipedia.page('coronavirus').content
print(wikipage)

Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds. In humans and birds, they cause respiratory tract infections that can range from mild to lethal. Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS, and COVID-19. In cows and pigs they cause diarrhea, while in mice they cause hepatitis and encephalomyelitis.
Coronaviruses constitute the subfamily Orthocoronavirinae, in the family Coronaviridae, order Nidovirales, and realm Riboviria. They are enveloped viruses with a positive-sense single-stranded RNA genome and a nucleocapsid of helical symmetry. The genome size of coronaviruses ranges from approximately 26 to 32 kilobases, one of the largest among RNA viruses. They have characteristic club-shaped spikes that project from their surface, which in electron micrographs create an image reminiscent of the solar corona, fro

In [13]:
wiki_sent = sent_tokenize(wikipage)
print(wiki_sent)

['Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds.', 'In humans and birds, they cause respiratory tract infections that can range from mild to lethal.', 'Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS, and COVID-19.', 'In cows and pigs they cause diarrhea, while in mice they cause hepatitis and encephalomyelitis.', 'Coronaviruses constitute the subfamily Orthocoronavirinae, in the family Coronaviridae, order Nidovirales, and realm Riboviria.', 'They are enveloped viruses with a positive-sense single-stranded RNA genome and a nucleocapsid of helical symmetry.', 'The genome size of coronaviruses ranges from approximately 26 to 32 kilobases, one of the largest among RNA viruses.', 'They have characteristic club-shaped spikes that project from their surface, which in electron micrographs create an image reminiscent o

In [15]:
wiki_words = []
for sent in wiki_sent:
    wiki_words.extend(word_tokenize(sent))
print(wiki_words)

['Coronaviruses', 'are', 'a', 'group', 'of', 'related', 'RNA', 'viruses', 'that', 'cause', 'diseases', 'in', 'mammals', 'and', 'birds', '.', 'In', 'humans', 'and', 'birds', ',', 'they', 'cause', 'respiratory', 'tract', 'infections', 'that', 'can', 'range', 'from', 'mild', 'to', 'lethal', '.', 'Mild', 'illnesses', 'in', 'humans', 'include', 'some', 'cases', 'of', 'the', 'common', 'cold', '(', 'which', 'is', 'also', 'caused', 'by', 'other', 'viruses', ',', 'predominantly', 'rhinoviruses', ')', ',', 'while', 'more', 'lethal', 'varieties', 'can', 'cause', 'SARS', ',', 'MERS', ',', 'and', 'COVID-19', '.', 'In', 'cows', 'and', 'pigs', 'they', 'cause', 'diarrhea', ',', 'while', 'in', 'mice', 'they', 'cause', 'hepatitis', 'and', 'encephalomyelitis', '.', 'Coronaviruses', 'constitute', 'the', 'subfamily', 'Orthocoronavirinae', ',', 'in', 'the', 'family', 'Coronaviridae', ',', 'order', 'Nidovirales', ',', 'and', 'realm', 'Riboviria', '.', 'They', 'are', 'enveloped', 'viruses', 'with', 'a', 'posi

In [16]:
tagged = pos_tag(wiki_words)
print(tagged)

[('Coronaviruses', 'NNS'), ('are', 'VBP'), ('a', 'DT'), ('group', 'NN'), ('of', 'IN'), ('related', 'JJ'), ('RNA', 'NNP'), ('viruses', 'VBZ'), ('that', 'IN'), ('cause', 'NN'), ('diseases', 'NNS'), ('in', 'IN'), ('mammals', 'NNS'), ('and', 'CC'), ('birds', 'NNS'), ('.', '.'), ('In', 'IN'), ('humans', 'NNS'), ('and', 'CC'), ('birds', 'NNS'), (',', ','), ('they', 'PRP'), ('cause', 'VBP'), ('respiratory', 'JJ'), ('tract', 'NN'), ('infections', 'NNS'), ('that', 'WDT'), ('can', 'MD'), ('range', 'VB'), ('from', 'IN'), ('mild', 'NN'), ('to', 'TO'), ('lethal', 'VB'), ('.', '.'), ('Mild', 'NNP'), ('illnesses', 'VBZ'), ('in', 'IN'), ('humans', 'NNS'), ('include', 'VBP'), ('some', 'DT'), ('cases', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('common', 'JJ'), ('cold', 'NN'), ('(', '('), ('which', 'WDT'), ('is', 'VBZ'), ('also', 'RB'), ('caused', 'VBN'), ('by', 'IN'), ('other', 'JJ'), ('viruses', 'NNS'), (',', ','), ('predominantly', 'RB'), ('rhinoviruses', 'VBZ'), (')', ')'), (',', ','), ('while', 'IN'), (

In [17]:
from nltk.corpus import wordnet

def getpos(tag):
    if(tag.startswith('J')):
        return wordnet.ADJ
    elif(tag.startswith('V')):
        return wordnet.VERB
    elif(tag.startswith('N')):
        return wordnet.NOUN
    elif(tag.startswith('R')):
        return wordnet.ADV
    else:
        return ''

In [19]:
from string import punctuation
stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()
lemma_pos = []
for word, tag in tagged:
    if(word not in punctuation and word not in stop_words and word.isalpha()):
        p = getpos(tag)
        if(p != ''):
            lemma = lemmatizer.lemmatize(word, pos=p)
            lemma_pos.append(lemma)
print(lemma_pos)

['Coronaviruses', 'group', 'related', 'RNA', 'viruses', 'cause', 'disease', 'mammal', 'bird', 'human', 'bird', 'cause', 'respiratory', 'tract', 'infection', 'range', 'mild', 'lethal', 'Mild', 'illnesses', 'human', 'include', 'case', 'common', 'cold', 'also', 'cause', 'virus', 'predominantly', 'rhinoviruses', 'lethal', 'variety', 'cause', 'SARS', 'MERS', 'cow', 'pig', 'cause', 'diarrhea', 'mouse', 'cause', 'hepatitis', 'encephalomyelitis', 'Coronaviruses', 'constitute', 'subfamily', 'Orthocoronavirinae', 'family', 'Coronaviridae', 'order', 'Nidovirales', 'realm', 'Riboviria', 'enveloped', 'virus', 'RNA', 'genome', 'nucleocapsid', 'helical', 'symmetry', 'genome', 'size', 'coronaviruses', 'range', 'approximately', 'kilobases', 'large', 'RNA', 'virus', 'characteristic', 'spike', 'project', 'surface', 'electron', 'micrographs', 'create', 'image', 'reminiscent', 'solar', 'corona', 'name', 'derive', 'Etymology', 'name', 'coronavirus', 'derive', 'Latin', 'corona', 'mean', 'crown', 'wreath', 'b

In [20]:
id2word = corpora.Dictionary([lemma_pos])

In [30]:
corpus = [id2word.doc2bow([text]) for text in lemma_pos]

In [31]:
corpus

[[(46, 1)],
 [(508, 1)],
 [(754, 1)],
 [(158, 1)],
 [(929, 1)],
 [(295, 1)],
 [(387, 1)],
 [(613, 1)],
 [(270, 1)],
 [(527, 1)],
 [(270, 1)],
 [(295, 1)],
 [(769, 1)],
 [(885, 1)],
 [(546, 1)],
 [(735, 1)],
 [(630, 1)],
 [(590, 1)],
 [(123, 1)],
 [(533, 1)],
 [(527, 1)],
 [(539, 1)],
 [(288, 1)],
 [(324, 1)],
 [(318, 1)],
 [(239, 1)],
 [(295, 1)],
 [(928, 1)],
 [(710, 1)],
 [(775, 1)],
 [(590, 1)],
 [(918, 1)],
 [(295, 1)],
 [(171, 1)],
 [(115, 1)],
 [(353, 1)],
 [(697, 1)],
 [(295, 1)],
 [(378, 1)],
 [(641, 1)],
 [(295, 1)],
 [(517, 1)],
 [(421, 1)],
 [(46, 1)],
 [(338, 1)],
 [(843, 1)],
 [(144, 1)],
 [(460, 1)],
 [(44, 1)],
 [(669, 1)],
 [(135, 1)],
 [(743, 1)],
 [(167, 1)],
 [(434, 1)],
 [(928, 1)],
 [(158, 1)],
 [(501, 1)],
 [(661, 1)],
 [(513, 1)],
 [(859, 1)],
 [(501, 1)],
 [(813, 1)],
 [(350, 1)],
 [(735, 1)],
 [(248, 1)],
 [(577, 1)],
 [(583, 1)],
 [(158, 1)],
 [(928, 1)],
 [(300, 1)],
 [(826, 1)],
 [(724, 1)],
 [(853, 1)],
 [(412, 1)],
 [(628, 1)],
 [(354, 1)],
 [(534, 1)],
 [

In [37]:
lda_model = models.LdaModel(corpus=corpus,
                            id2word=id2word,
                            num_topics = 3,
                            random_state=100,
                            chunksize=100,
                            passes=10,
                            per_word_topics=True)
print(lda_model.print_topics())

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, '0.092*"coronavirus" + 0.040*"herd" + 0.023*"Coronavirus" + 0.021*"human" + 0.020*"References" + 0.020*"entire" + 0.020*"Further" + 0.020*"Zoonosis" + 0.020*"destruction" + 0.020*"prevent"'), (1, '0.050*"virus" + 0.031*"also" + 0.031*"pig" + 0.029*"specie" + 0.029*"use" + 0.026*"coronaviruses" + 0.023*"reading" + 0.023*"animal" + 0.020*"respiratory" + 0.016*"disease"'), (2, '0.043*"cause" + 0.027*"transmission" + 0.025*"infect" + 0.019*"bat" + 0.018*"strain" + 0.016*"diarrhea" + 0.015*"case" + 0.015*"different" + 0.011*"bronchitis" + 0.011*"high"')]


In [38]:
!pip install pyLDAvis

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
import pyLDAvis.gensim

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
vis

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.054171 -0.197628       1        1  34.189297
1     -0.157077  0.142413       2        1  33.489488
0      0.211248  0.055215       3        1  32.321215, topic_info=            Term       Freq      Total Category  logprob  loglift
349  coronavirus  75.000000  75.000000  Default  30.0000  30.0000
928        virus  42.000000  42.000000  Default  29.0000  29.0000
295        cause  37.000000  37.000000  Default  28.0000  28.0000
518         herd  33.000000  33.000000  Default  27.0000  27.0000
239         also  27.000000  27.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
158          RNA   4.596612   5.112444   Topic3  -5.1761   1.0231
383       direct   4.217668   4.738797   Topic3  -5.2621   1.0129
293       cattle   4.675295   5.263789   Topic3  -5.1591   1.0109
800       severe   3.956353   4.492414   Topic3  -5.3261   1.0024
720   production   3.995398   4.744141   Topic3  -5.3162   0.9577

[124 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
5         2  0.914723  Alphacoronavirus
21        2  0.904769   Betacoronavirus
32        1  0.901511            Canine
45        3  0.973868       Coronavirus
46        3  0.932609     Coronaviruses
...     ...       ...               ...
913       2  0.972132               use
914       2  0.974195           vaccine
925       2  0.939971             viral
928       2  0.994840             virus
946       2  0.865719             young

[103 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

##### Gerador de Resumos

In [43]:
!pip install wikipedia

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [91]:
import wikipedia
wikipage = wikipedia.page('Brazil').content
print(wikipage)

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw]), officially the Federative Republic of Brazil (Portuguese: República Federativa do Brasil), is the largest country in both South America and Latin America. At 8.5 million square kilometers (3.2 million square miles) and with over 211 million people, Brazil is the world's fifth-largest country by area and the sixth most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the largest country to have Portuguese as an official language and the only one in the Americas; it is also one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world; as well as the most populous Roman Catholic-majority country.
Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi). It borders all other countries in South America except Ecu

In [92]:
import nltk
from nltk.tokenize import sent_tokenize, PunktSentenceTokenizer
nltk.download('punkt')

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [93]:
#wiki_sent = sent_tokenize(wikipage)
sent_tokenizer = PunktSentenceTokenizer(wikipage)
sentences = sent_tokenizer.tokenize(wikipage)

wiki_sent = []
for sentence in sentences:
    wiki_sent.extend(sentence.split('\n'))
#print(wiki_sent)

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
from nltk.tokenize import word_tokenize

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [95]:
wiki_words = []
for sent in wiki_sent:
    wiki_words.extend(word_tokenize(sent))
#print(wiki_words)

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [96]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
word_frequencies = {}
for word in wiki_words:
    if(word not in stopwords and word.isalpha()):
        if(word not in word_frequencies.keys()):
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1
#print(word_frequencies)

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [98]:
maximum_frequency = max(word_frequencies.values())

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [99]:
maximum_frequency

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


193

In [100]:
for word in word_frequencies.keys():
    word_frequencies[word] /= maximum_frequency
#print(word_frequencies)

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [104]:
sentence_scores = {}
for sent in wiki_sent:
    for word in word_tokenize(sent):
        if word in word_frequencies.keys():
            if(sent not in sentence_scores.keys()):
                sentence_scores[sent] = word_frequencies[word]
            else:
                sentence_scores[sent] += word_frequencies[word]
for sent in sentence_scores.keys():
    print(sent, '------>', sentence_scores[sent])

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw]), officially the Federative Republic of Brazil (Portuguese: República Federativa do Brasil), is the largest country in both South America and Latin America. ------> 4.709844559585494
At 8.5 million square kilometers (3.2 million square miles) and with over 211 million people, Brazil is the world's fifth-largest country by area and the sixth most populous. ------> 2.6217616580310876
Its capital is Brasília, and its most populous city is São Paulo. ------> 0.34196891191709844
The federation is composed of the union of the 26 states and the Federal District. ------> 0.9015544041450778
It is the largest country to have Portuguese as an official language and the only one in the Americas; it is also one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world; as well as the most populous Roman Catholic-majority country. ------> 3.0259067357512954
Bounded by the Atlant

In [109]:
import heapq
summary_sentences = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)
sorted_summary = [sentence for sentence in wiki_sent if sentence in summary_sentences]
print(sorted_summary)

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw]), officially the Federative Republic of Brazil (Portuguese: República Federativa do Brasil), is the largest country in both South America and Latin America.', 'Throughout the 16th century, massive amounts of brazilwood were harvested by indigenous peoples (mostly Tupi) along the Brazilian coast, who sold the timber to European traders (mostly Portuguese, but also French) in return for assorted European consumer goods.The official Portuguese name of the land, in original Portuguese records, was the "Land of the Holy Cross" (Terra da Santa Cruz), but European sailors and merchants commonly called it simply the "Land of Brazil" (Terra do Brasil) because of the brazilwood trade.', "In this era other European powers tried to colonize parts of Brazil, in incursions that the Portuguese had to fight, notably the French in Rio during the 1560s, in Maranhão during the 1610s, and the Dutch in Bahia and Pernambuco, during the Dutch–Port

In [110]:
summary = ' '.join(sorted_summary)
print(summary)

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw]), officially the Federative Republic of Brazil (Portuguese: República Federativa do Brasil), is the largest country in both South America and Latin America. Throughout the 16th century, massive amounts of brazilwood were harvested by indigenous peoples (mostly Tupi) along the Brazilian coast, who sold the timber to European traders (mostly Portuguese, but also French) in return for assorted European consumer goods.The official Portuguese name of the land, in original Portuguese records, was the "Land of the Holy Cross" (Terra da Santa Cruz), but European sailors and merchants commonly called it simply the "Land of Brazil" (Terra do Brasil) because of the brazilwood trade. In this era other European powers tried to colonize parts of Brazil, in incursions that the Portuguese had to fight, notably the French in Rio during the 1560s, in Maranhão during the 1610s, and the Dutch in Bahia and Pernambuco, during the Dutch–Portuguese W

### Analise de Sentimentos

In [111]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
import nltk

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
nltk.download('punkt')
nltk.download('vader_lexicon')

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [113]:
string = 'I read a book. The first three chapters were boring and depressive. Then the next 2 chapters were wonderful. I recommend this book to my friends.'

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [114]:
sent = sent_tokenize(string)
sent

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['I read a book.',
 'The first three chapters were boring and depressive.',
 'Then the next 2 chapters were wonderful.',
 'I recommend this book to my friends.']

In [115]:
sa = SentimentIntensityAnalyzer()
for sentence in sent:
    print(sentence)
    ps = sa.polarity_scores(sentence)
    for n in ps:
        print(n,':',ps[n])
        print()

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


I read a book.
neg : 0.0

neu : 1.0

pos : 0.0

compound : 0.0

The first three chapters were boring and depressive.
neg : 0.45

neu : 0.55

pos : 0.0

compound : -0.5994

Then the next 2 chapters were wonderful.
neg : 0.0

neu : 0.575

pos : 0.425

compound : 0.5719

I recommend this book to my friends.
neg : 0.0

neu : 0.417

pos : 0.583

compound : 0.6808



In [116]:
import pandas as pd

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [118]:
data = pd.read_csv('Tweets_Mg.csv')

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [120]:
data['Text']

C:\Users\Lucas\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0       ���⛪ @ Catedral de Santo Antônio - Governador ...
1       � @ Governador Valadares, Minas Gerais https:/...
2       �� @ Governador Valadares, Minas Gerais https:...
3                             ��� https://t.co/BnDsO34qK0
4       ��� PSOL vai questionar aumento de vereadores ...
                              ...                        
8194    Trio é preso suspeito de roubo, tráfico e abus...
8195    Trio é preso suspeito de roubo, tráfico e abus...
8196    Trio é preso suspeito de roubo, tráfico e abus...
8197    Trio é preso suspeito de roubo, tráfico e abus...
8198    Trio suspeito de roubo de cargas é preso em Sa...
Name: Text, Length: 8199, dtype: object